# Machine learning

In [4]:
import json
import requests
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
df = pd.read_csv("example_data2.csv")

## Kernel density estimation

In [5]:
def cvt2date(df):
    rst = []
    for i in range(len(df)):
        rst.append(df['datetime'].iloc[i].split()[0])
    df['date'] = rst
    df['date'] =pd.to_datetime(df.date)
    df = df.sort_values(by='date',ascending=True)

df_wf = df[df['title']=='Wildfires']
cvt2date(df_wf)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
from sklearn.neighbors import KernelDensity
from datetime import datetime

def norm_kde(df_wf):
    mean_geo2 = np.mean(df_wf['geo2'])
    mean_geo1 = np.mean(df_wf['geo1'])
    mean_date = np.mean([datetime.fromisoformat(str(df_wf['date'].iloc[i])).timestamp() for i in range(len(df_wf))])

    std_geo2 = np.std(df_wf['geo2'])
    std_geo1 = np.std(df_wf['geo1'])
    std_date = np.std([datetime.fromisoformat(str(df_wf['date'].iloc[i])).timestamp() for i in range(len(df_wf))])
    
    X = np.array([ [ (df_wf['geo2'].iloc[i] - mean_geo2)/std_geo2,
                    (df_wf['geo1'].iloc[i] - mean_geo1)/std_geo1,
                    (datetime.fromisoformat(str(df_wf['date'].iloc[i])).timestamp()-mean_date)/std_date ] 
                  for i in range(len(df_wf)) ])
    kde = KernelDensity(kernel='gaussian', bandwidth=.1).fit(X)
    return kde,(mean_geo2,mean_geo1,mean_date,std_geo2,std_geo1,std_date)
kde = norm_kde(df_wf)

def kde_predict(x,kde):
    a = (x[0]-kde[1][0])/kde[1][3]
    b = (x[1]-kde[1][1])/kde[1][4]
    c = (x[2]-kde[1][2])/kde[1][5]
    return kde[0].score_samples([[a,b,c]])[0]
    
kde_pred = [kde_predict([df_wf['geo2'].iloc[i],
                    df_wf['geo1'].iloc[i],
                    datetime.fromisoformat(str(df_wf['date'].iloc[i])).timestamp()]
                   ,kde
                  )  for i in range(len(df_wf))]

## Random forest

In [7]:
df_la = pd.read_csv('la_history_w.csv')
# la 34 -118
# st 47 -122
del df_la['Unnamed: 0']
df_la['date'] = [datetime.fromisoformat(str(df_la['date'].iloc[i])).timestamp() for i in range(len(df_la))]
df_la.head()
y = [kde_predict([34,-118,i],kde) for i in df_la['date']]
X = df_la

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

cont_ftr = df_la.columns[0:5].tolist() + df_la.columns[6:30].tolist()

def rf_pip(X,y,random_state,n_folds):
    
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.3, random_state = random_state[0])
    kf = KFold(n_splits=n_folds,shuffle=True,random_state=random_state[1])
    CV_scores = []
    test_scores = []
    
    for train_index, CV_index in kf.split(X_other,y_other):
        
        X_train, X_CV = X_other.iloc[train_index,:], X_other.iloc[CV_index,:]
        y_other = np.array(y_other)
        y_train, y_CV = y_other[train_index], y_other[CV_index]

        X_train_cont = X_train[cont_ftr]
        X_train = X_train_cont

        X_c_cont = X_CV[cont_ftr]
        X_c = X_c_cont

        X_t_cont = X_test[cont_ftr]
        X_t = X_t_cont

        train_score = []
        CV_score = []
        regs = []
        par = range(2,11,2)
        par_ls = []
        for i in par:
            for j in par:
                reg = RandomForestRegressor(random_state = random_state[2], max_depth=i, 
                                             min_samples_split=j,n_estimators = 100)
        reg.fit(X_train,y_train)
        train_score.append(r2_score(y_train,reg.predict(X_train)))
        CV_score.append(r2_score(y_CV,reg.predict(X_c)))
        regs.append(reg)
        best_par = (par[np.argmax(CV_score)//len(par)],par[np.argmin(CV_score)%len(par)])
        reg = regs[np.argmax(CV_score)]
        CV_scores.append(np.max(CV_score))
        test_scores.append(r2_score(y_test,reg.predict(X_t)))
    return (CV_scores,test_scores,best_par)#

In [9]:
ac = []
for i in range(1,6):
    op = rf_pip(X,y,[10*i,14*i,96*i],5)
    ac.append(np.mean(op[1]))
print("Mean of test r2 scores: " + str(np.mean(ac)))
print("Std of test r2 scores: " + str(np.std(ac)))

Mean of test r2 scores: 0.7720578592210845
Std of test r2 scores: 0.014162746703444612


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pickle


X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.4,random_state=1)
X_train_cont = X_other[cont_ftr]

X_t_cont = X_test[cont_ftr]

reg = RandomForestRegressor(max_depth=2,min_samples_split=2,random_state=4)
reg.fit(X_train_cont,y_other)
# pickle.dump(reg, open("rf_002_la.pickle", 'wb'))
r2_score(y_test,reg.predict(X_t_cont))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



0.6645163653871835

In [11]:
pd.DataFrame({'ftr':cont_ftr,'importance':reg.feature_importances_})

,ftr,importance
0,apparentTemperatureHigh,0.126842
1,apparentTemperatureLow,0.070695
2,apparentTemperatureMax,0.008520
3,apparentTemperatureMin,0.158102
4,cloudCover,0.014565
5,dewPoint,0.000000
6,humidity,0.000000
7,lat,0.000000
8,long,0.000000
9,moonPhase,0.000000


## Conclusion

- Weather data can be used to predict fire alarm rate.
- Temperature is an important feature in fire alarm rate prediction.